In [1]:
import pickle
from pathlib import Path

import numpy as np
import pyvista as pv
import trimesh as tm

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import construction as ulac

In [2]:
def plot_over_dict(plotter, mesh, input_value, transformation, color, size):
    if isinstance(input_value, dict):
        for value in input_value.values():
            plot_over_dict(plotter, mesh, value, transformation, color, size)
    else:
        indices, scalars = transformation(input_value)
        point_mesh = pv.PolyData(mesh.points[indices])
        if scalars is not None:
            point_mesh["scalars"] = scalars
            plotter.add_mesh(
                point_mesh,
                scalars="scalars",
                render_points_as_spheres=True,
                point_size=size,
            )
        else:
            plotter.add_mesh(
                point_mesh,
                color=color,
                render_points_as_spheres=True,
                point_size=size,
            )


def plot_uacs_over_dict(plotter, mesh, input_value):
    if isinstance(input_value, dict):
        for value in input_value.values():
            plot_uacs_over_dict(plotter, mesh, value)
    else:
        mapping_3d = np.hstack(
            (
                input_value.alpha[:, None],
                input_value.beta[:, None],
                np.zeros((input_value.alpha.shape[0], 1)),
            )
        )
        uac_mesh = pv.PolyData.from_regular_faces(mapping_3d, input_value.connectivity)
        plotter.add_mesh(uac_mesh, style="wireframe", color="gray")

In [3]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = {"MV": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [4]:
if segmentation_file_name.exists():
    with segmentation_file_name.open("rb") as f:
        segmentation_paths = pickle.load(f)
else:
    segmentation_paths = ulac.construct_segmentation(mesh, anatomical_tags)
    segmentation_file_name.parent.mkdir(parents=True, exist_ok=True)
    with segmentation_file_name.open("wb") as f:
        pickle.dump(segmentation_paths, f)

transformation = lambda input_value: (input_value, None)
plotter = pv.Plotter(window_size=[700, 500])
plotter.add_mesh(mesh, style="surface", show_edges=True, color="grey")

plot_over_dict(plotter, mesh, segmentation_paths, transformation, "blue", 15)
plotter.add_points(mesh.points[[12959,12966,12967]], color="yellow", point_size=15, render_points_as_spheres=True)
#plotter.add_points(mesh.points[[12961,12967]], color="red", point_size=15, render_points_as_spheres=True)
#plotter.add_points(mesh.points[[12967,12975]], color="green", point_size=15, render_points_as_spheres=True)


plotter.show()

Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f2195006510_0&reconnect=auto" class="pyvi…

In [5]:
markers = ulac.get_markers(segmentation_paths)

plotter = pv.Plotter(window_size=[700, 500])
plotter.add_mesh(mesh, style="wireframe", color="grey")
transformation = lambda input_value: (input_value, None)
plot_over_dict(plotter, mesh, segmentation_paths, transformation, "blue", 5)
transformation = lambda input_value: (np.array((input_value.ind,)), None)
plot_over_dict(plotter, mesh, markers, transformation, "red", 10)
plotter.show()

Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f217defe210_1&reconnect=auto" class="pyvi…

In [6]:
parameterized_paths = ulac.parameterize_paths(mesh, segmentation_paths, markers)

plotter = pv.Plotter(window_size=[700, 500])
plotter.add_mesh(mesh, style="surface", show_edges=True, color="grey")
transformation = lambda input_value: (input_value.inds, input_value.relative_lengths)
plot_over_dict(plotter, mesh, parameterized_paths, transformation, "blue", 15)
plotter.show()

Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f215935c050_2&reconnect=auto" class="pyvi…

In [7]:
uac_paths = ulac.construct_uac_paths(parameterized_paths, markers)

plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 500])
plotter.subplot(0, 0)
plotter.add_mesh(mesh, style="wireframe", show_edges=True, color="grey")
transformation = lambda input_value: (input_value.inds, input_value.alpha)
plot_over_dict(plotter, mesh, uac_paths, transformation, "blue", 10)
plotter.subplot(0, 1)
plotter.add_mesh(mesh, style="wireframe", show_edges=True, color="grey")
transformation = lambda input_value: (input_value.inds, input_value.beta)
plot_over_dict(plotter, mesh, uac_paths, transformation, "blue", 10)
plotter.show()

Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f215935de50_3&reconnect=auto" class="pyvi…

In [68]:
patch_boundaries = ulac.get_patch_boundaries(uac_paths)
uac_patches = ulac.compute_patch_uacs(mesh, patch_boundaries, segmentation_paths)

plotter = pv.Plotter(window_size=[900, 900])
plot_uacs_over_dict(plotter, mesh, uac_patches)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f214056c190_60&reconnect=auto" class="pyv…

In [9]:
alpha_list = []
beta_list = []
simplex_list = []
global num_points
num_points = 0


def collect_uac_data(input_value):
    global num_points
    if isinstance(input_value, dict):
        for value in input_value.values():
            collect_uac_data(value)
    else:
        alpha_list.append(input_value.alpha)
        beta_list.append(input_value.beta)
        simplex_list.append(input_value.connectivity + num_points)
        num_points += input_value.inds.size


collect_uac_data(uac_patches)
alpha_values = np.concatenate(alpha_list)
beta_values = np.concatenate(beta_list)
uacs = np.vstack((alpha_values, beta_values)).T
uacs = np.hstack((uacs, np.zeros((uacs.shape[0], 1))))
simplices = np.vstack(simplex_list)

In [67]:
uac_mesh = pv.PolyData.from_regular_faces(uacs, simplices)
uac_mesh = uac_mesh.clean(strips_to_polys=True)
uac_mesh = uac_mesh.smooth(n_iter=300, relaxation_factor=0.001, edge_angle=360, feature_angle=360)
#uac_mesh = uac_mesh.smooth_taubin(n_iter=10000, pass_band=1, feature_angle=180, edge_angle=180)

tm_mesh = tm.Trimesh(uac_mesh.points, uac_mesh.faces.reshape(-1, 4)[:, 1:], process=False)
print(np.min(np.sum(tm_mesh.face_normals, axis=1)))

plotter = pv.Plotter(window_size=[700, 700])
plotter.add_mesh(uac_mesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()

0.9999999999999999


Widget(value='<iframe src="http://localhost:40971/index.html?ui=P_0x7f1faa65e350_59&reconnect=auto" class="pyv…